In [1]:
# hide
# default_exp params

# Notebook Parameter Management

Having the ability to parameterise notebook executions is very useful..

# Notebook utilities 

> Used to create & access the parameters cell.

In [82]:
# export

from io import StringIO
from pathlib import Path
from typing import Iterable

import nbformat
from nbdev.export import Config, find_default_export, nbglob, read_nb
from nbformat.notebooknode import NotebookNode

In [83]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# export


def find_params_cell(nb: NotebookNode):
    params_cell = [c for c in nb["cells"] if c["metadata"] == {"tags": ["parameters"]}]
    return params_cell

In [11]:
import os

test_nb = os.path.join(Path(".").resolve(), "test", "test_export.ipynb")

In [12]:
assert len(find_params_cell(read_nb(Path(test_nb)))) == 1
assert len(find_params_cell(read_nb(Path("index.ipynb")))) == 0

In [96]:
# export


def extract_params(nb: NotebookNode):
    params_cell = find_params_cell(nb)
    return params_cell[0]["source"] if len(params_cell) > 0 else None

In [25]:
print(extract_params(read_nb(Path(test_nb))))

# export

some_params = [1,2,3]
some_params = 'test'
input_path = Path('.')
model_path = Path('.').resolve().parent


In [28]:
params_code = extract_params(read_nb(Path(test_nb)))
assert params_code.startswith("# export")
assert "some_param" in params_code
assert "some_params" in params_code
assert "input_path" in params_code
assert "model_path" in params_code

In [15]:
# export

DEFAULT_PARAMS_CELL = {
    "cell_type": "code",
    "execution_count": None,
    "metadata": {"tags": ["parameters"]},
    "outputs": [],
    "source": "# parameters\n",
}

In [16]:
# export


def add_missing_params_cell(nb_path: Path, persist: bool = True):
    nb = read_nb(nb_path)
    if len(find_params_cell(nb)) > 0:
        print(f"Skipping {nb_path} already has parameters cell")
        return
    nb["cells"].insert(0, nbformat.from_dict(DEFAULT_PARAMS_CELL))
    if persist:
        nbformat.write(nb, nb_path)
    return nb

In [17]:
with_params = os.path.join(Path(".").resolve().parent, "examples", "top2vec.ipynb")
without_params = os.path.join(
    Path(".").resolve().parent, "examples", "top2vec-no-params.ipynb"
)

add_missing_params_cell(with_params, False)
assert len(find_params_cell(read_nb(without_params))) == 0
parameterised_nb = add_missing_params_cell(without_params, False)
assert len(find_params_cell(parameterised_nb)) == 1

Skipping /home/jovyan/git/sciflow/examples/top2vec.ipynb already has parameters cell


In [29]:
# export


def extract_params_to_file(nb_path: Path, params_file_path: Path):
    params_code = extract_params(read_nb(Path(test_nb)))
    with open(params_file_path, "w") as params_file:
        params_file.writelines(params_code)

In [59]:
extract_params_to_file(
    test_nb, os.path.join(Config().path("lib_path"), "test_export_params.py")
)

In [60]:
# export


def list_mod_files(files):
    modules = []
    for f in files:
        fname = Path(f)
        nb = read_nb(fname)
        default = find_default_export(nb["cells"])
        if default is not None:
            default = os.path.sep.join(default.split("."))
            modules.append(default)
    return modules

In [61]:
# export


def extract_as_files(suffix="_params.py"):
    nbs = nbglob(recursive=True)
    param_files = list_mod_files(nbs)
    params_files = [
        Path(os.path.join(Config().path("lib_path"), pf + suffix)) for pf in param_files
    ]
    for nb_path, pf_path in zip(nbs, params_files):
        extract_params_to_file(nb_path, pf_path)

In [62]:
# exporti


def _lines_to_dict(lines: Iterable[str]):
    result = {}
    for line in lines:
        if line.startswith("#") or not "=" in line:
            continue
        (key, val) = line.split("=")
        result[key.strip()] = val.strip('\n "')
    return result

In [63]:
# export


def extract_params_as_dict(params_file_path: Path):
    params = {}
    with open(params_file_path, "r") as params_file:
        params = _lines_to_dict(params_file.readlines())
    return params

In [68]:
params_dict = extract_params_as_dict(
    os.path.join(Config().path("lib_path"), "test_export_params.py")
)
tup = tuple(params_dict.keys())

In [97]:
# export


def params_as_dict(nb_path: Path):
    params_code = extract_params(read_nb(nb_path))
    params = _lines_to_dict(StringIO(params_code).readlines())
    return params

In [99]:
assert ["input_path", "model_path", "some_param", "some_params"] == list(
    sorted(params_as_dict(test_nb).keys())
)